In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from datetime import date
from datetime import timedelta

from yahoofinancials import YahooFinancials

from scipy.stats import norm

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

import warnings
warnings.filterwarnings('ignore')

In [2]:
end = '2021-09-25'
start = '2016-07-20'

In [3]:
stocks = ['IRCTC', 'INFY', 'TATACONSUM', 'HINDUNILVR']
portfolio = stocks.copy()

portfolio_ticker = []
for stock in stocks:
    portfolio_ticker.append(stock+'.NS')
    
stocks += ['NIFTY']
portfolio_ticker += ['^NSEI']

In [4]:
portfolio

['IRCTC', 'INFY', 'TATACONSUM', 'HINDUNILVR']

In [5]:
for stock_name, stock_ticker in zip(stocks, portfolio_ticker):
    
    yf = YahooFinancials(stock_ticker)
    data = yf.get_historical_price_data(start, end, 'daily')
    globals()[stock_name] = pd.DataFrame(data[stock_ticker]['prices'])
    globals()[stock_name] = globals()[stock_name].drop('date', axis=1).set_index('formatted_date')

    globals()[stock_name]['Date'] = pd.to_datetime(globals()[stock_name].index, format="%Y-%m-%d")
    globals()[stock_name].set_index('Date', drop=False, inplace=True)

    globals()[stock_name] = globals()[stock_name].dropna()

In [6]:
closing_df = pd.DataFrame(index=INFY.index, columns=stocks)
for stock in stocks:
    closing_df[stock] = globals()[stock]['close']

In [7]:
closing_df['formatted_date'] = INFY['Date']

In [8]:
closing_df

,IRCTC,INFY,TATACONSUM,HINDUNILVR,NIFTY,formatted_date
Date,,,,,,
2016-07-20,NaN,541.549988,133.850006,906.900024,8565.849609,2016-07-20
2016-07-21,NaN,539.950012,131.800003,910.049988,8510.099609,2016-07-21
2016-07-22,NaN,536.224976,131.500000,903.200012,8541.200195,2016-07-22
2016-07-25,NaN,540.424988,133.449997,906.950012,8635.650391,2016-07-25
2016-07-26,NaN,544.349976,131.149994,902.700012,8590.650391,2016-07-26
...,...,...,...,...,...,...
2021-09-20,3707.899902,1687.849976,857.000000,2800.350098,17396.900391,2021-09-20
2021-09-21,3624.550049,1718.449951,858.900024,2812.449951,17562.000000,2021-09-21
2021-09-22,3671.300049,1716.900024,854.150024,2784.500000,17546.650391,2021-09-22


In [9]:
from prophet import Prophet

In [10]:
pred_df = pd.DataFrame(columns=stocks)

In [11]:
for stock_name in stocks:
    
    df = closing_df[['formatted_date', stock_name]].copy()
    df.reset_index(drop=True, inplace=True)

    df.rename(columns={"formatted_date": "ds", stock_name: "y"}, inplace=True)
    df['ds'] = pd.to_datetime(df['ds'], errors='coerce')
    
    model=Prophet(changepoint_prior_scale= 0.4, n_changepoints=100, seasonality_mode='multiplicative') 
    model.fit(df)

    future = model.make_future_dataframe(periods=90, freq='B')
    forecast = model.predict(future)
    
    pred_df[stock_name] = forecast['yhat']
    pred_df['date'] = forecast['ds']

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
pred_df

,IRCTC,INFY,TATACONSUM,HINDUNILVR,NIFTY,date
0,-7807.453044,542.719769,135.242672,920.049943,8638.817006,2016-07-20
1,-7792.243127,541.552936,135.214330,916.043026,8630.597210,2016-07-21
2,-7759.164037,541.282325,135.205638,913.813176,8624.084576,2016-07-22
3,-7722.152808,539.935877,134.469459,906.733760,8593.427515,2016-07-25
4,-7768.296711,539.604318,134.683298,906.235549,8604.054347,2016-07-26
...,...,...,...,...,...,...
1363,4716.765208,2123.757978,892.370364,3025.274012,21044.391333,2022-01-24
1364,4761.449999,2117.433329,890.688445,3019.011032,21091.003264,2022-01-25
1365,4770.815222,2113.156571,887.417918,3010.793626,21109.896415,2022-01-26
1366,4777.996385,2103.271715,885.607506,2996.297199,21120.877401,2022-01-27


In [13]:
pred = pred_df[-90:]
pred.set_index('date', drop=True, inplace=True)

In [14]:
pred

,IRCTC,INFY,TATACONSUM,HINDUNILVR,NIFTY
date,,,,,
2021-09-27,3291.023742,1784.232272,832.383093,2803.330264,17589.119292
2021-09-28,3325.848177,1794.200749,829.079181,2807.413881,17620.569360
2021-09-29,3336.025003,1806.452748,824.049007,2809.969744,17626.292593
2021-09-30,3344.668862,1814.086947,819.989706,2806.938982,17624.141694
2021-10-01,3345.576972,1824.354451,815.930347,2808.517230,17625.472903
...,...,...,...,...,...
2022-01-24,4716.765208,2123.757978,892.370364,3025.274012,21044.391333
2022-01-25,4761.449999,2117.433329,890.688445,3019.011032,21091.003264
2022-01-26,4770.815222,2113.156571,887.417918,3010.793626,21109.896415


In [15]:
monthly_return = pred.resample('M').ffill().pct_change()
monthly_return

,IRCTC,INFY,TATACONSUM,HINDUNILVR,NIFTY
date,,,,,
2021-09-30,NaN,NaN,NaN,NaN,NaN
2021-10-31,0.100770,0.037537,-0.052628,0.006691,0.033386
2021-11-30,0.109775,0.003344,0.090184,0.015940,0.075423
2021-12-31,0.086185,0.090006,0.086811,0.086219,0.051483
2022-01-31,0.075740,0.018367,-0.039373,-0.042224,0.026316


In [16]:
monthly_closing_df = pred.resample('M').mean()
monthly_closing_df

,IRCTC,INFY,TATACONSUM,HINDUNILVR,NIFTY
date,,,,,
2021-09-30,3324.391446,1799.743179,826.375247,2806.913218,17615.030735
2021-10-31,3529.940750,1887.243459,776.530142,2824.597348,17851.757718
2021-11-30,3893.265994,1873.373558,803.885768,2805.771429,18918.476613
2021-12-31,4271.406767,1960.690644,889.909248,3033.477750,20101.524903
2022-01-31,4633.916952,2124.147216,913.542426,3076.767339,20946.281361


In [17]:
nifty_std = monthly_return['NIFTY'].std()

mstd = monthly_return.copy()
mstdl = mstd.corr(method='pearson')

In [18]:
monthly_stock_beta = pd.Series(index=stocks)

for stock in stocks:
    monthly_stock_beta[stock] = (mstdl['NIFTY'][stock] * mstd[stock].std()) / nifty_std

In [19]:
monthly_stock_beta

IRCTC         0.484253
INFY         -0.236324
TATACONSUM    3.132562
HINDUNILVR    1.141845
NIFTY         1.000000
dtype: float64

In [20]:
metrics = ['Return','Volatility','Sharpe Ratio', 'VaR', 'Portfolio Beta', 'Treynor Ratio', 'Jensen Alpha']
metrics_df = pd.DataFrame(columns=metrics)

In [21]:
nifty_50_daily = pred['NIFTY'].pct_change().dropna()

In [22]:
risk_free_rate = 0.0602
alpha = 0.01
days = 21
market_return = nifty_50_daily.mean() * days 

In [23]:
w1 = np.asarray([0.25, 0.25, 0.25, 0.25])

In [24]:
r1 = np.zeros(len(metrics))
sb1 = monthly_stock_beta[portfolio].copy()
pb1 = 0
tdf1 = monthly_return[portfolio].copy()
twl1 = w1
tcovdf1 = tdf1.cov()

pr1 = np.sum(tdf1.mean() * twl1) * days
pstd1 = np.sqrt(np.dot(twl1.T,np.dot(tcovdf1, twl1))) * np.sqrt(days)

r1[0] = pr1
r1[1] = pstd1

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r1[2] = (r1[0] - risk_free_rate) / r1[1]

# VaR => return - (Z * volatility)
r1[3] = abs(pr1 - (pstd1 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl1)):
    pb1 += twl1[j] * sb1[j]
r1[4] = pb1

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r1[5] = (r1[0] - risk_free_rate) / r1[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r1[6] = r1[0] - (risk_free_rate + r1[4] * (market_return - risk_free_rate))   

r1 = pd.Series(r1, index=metrics)
metrics_df = metrics_df.append(r1, ignore_index=True)

In [25]:
metrics_df

,Return,Volatility,Sharpe Ratio,VaR,Portfolio Beta,Treynor Ratio,Jensen Alpha
0,0.883763,0.169105,4.870126,0.490366,1.130584,0.728441,0.842543
